# CS 4496/7496 Character Animation (Fall 2023)
Copyright (c) Georgia Institute of Technology


---





# Project 0: Introduction (due 9/8/2023, Friday, 11:59 PM)
Google Colab (https://colab.research.google.com) is an interactive web framework that can include live codes, texts, equations, and visualizations. This will be the perfect framework for learning computer animation because you can write the code and visualize the results interactively!

The goal of this first project is simple: setting up your homework environment, running the example snippets, and making sure that the results are rendered okay.  **Please get in touch with the instructors if you have any trouble with the notebook.** Don't worry (at this point) if you do not understand the example codes in Section 2 and 3: they are just skeleton codes and not the main learning objectives of the class.

# 1. Numpy Practice
In this section, we will learn how to use numpy (https://numpy.org/), which is the most popular numerical computation framework in Python.

We begin by importing the required libraries.

In [ ]:
import numpy as np

As the first example, let's create two vectors and add them together. You can simply run the below code.

In [ ]:
a = np.array([1.0, -2.0])
b = np.array([3.0, 3.0])
c = a + b
print("the sum of %s and %s = %s" % (str(a), str(b), str(c)))

See? It is not too difficult. We can also repeat the same for two dimensional arrays. We will also check the validity of the code by adding an assertion statement.

In [ ]:
a = np.array([[2.0, 1.0], [3.0, 1.0]])
b = np.array([[4.0, 1.0], [-1.0, 1.0]])
print("a + b = \n", a + b)
assert(np.allclose(a + b, np.array([[6.0, 2.0], [2.0, 2.0]])))
print('pass the test!')

You often need multiply matrices. Please make sure the difference between element-wise multiplication (* operator) and matrix multiplication (@, or np.dot).

In [ ]:
print("Element-wise multiplication")
print(a * b)
print("Matrix multiplication (dot proudct)")
print(a @ b)

How about inversion?

In [ ]:
Ainv = np.linalg.inv(a)
print("Ainv = \n", Ainv)
assert(np.allclose(a @ Ainv, np.identity(2)))
assert(np.allclose(Ainv @ a, np.identity(2)))
print("pass all the tests!")

Finally, let's do some practice. Let's see if you can write a matrix multiplication by yourself. Please fill the rest of the code (hint: Numpy cheatsheet: https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Numpy_Python_Cheat_Sheet.pdf)
** **write your code below (40 pts)** **

In [ ]:
def matrix_mul(M1, M2):
  m1, n1 = M1.shape # fetch the shape of M1
  m2, n2 = M2.shape # fetch the shape of M2
  if n1 != m2:
    return None

  ret = np.zeros((m1, n2)) # place holder for the return

  # Begin your answer

  # End your answer
  return ret

A = np.random.rand(3, 2)
B = np.random.rand(2, 4)
print("A = \n", A)
print("B = \n", B)
print("correct answer = \n", A @ B)
print("your answer = \n", matrix_mul(A, B))
assert(np.allclose(A @ B, matrix_mul(A, B), 1e-3))
print("Congratulations! You pass the test.")

# 2. Matplotlib Rendering Test
In this section, we will test animated rendering with matplotlib.

We begin by importing the required libraries.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation, rc
from IPython.display import HTML

Then we set up the initial figure with the empty dataset. This will draw an empty figure, which is not animated yet.



In [ ]:
fig, ax = plt.subplots()
ax.set_xlim(( -2.0, 2.0))
ax.set_ylim((-2.0, 2.0))
line, = ax.plot([], [], lw=2)

We will describe the animation with the "init" and "animate" functions.

In [ ]:
# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return (line,)

# animation function. This is called sequentially
def animate(i):
    x = np.linspace(0, 2, 1000)
    y = np.sin(2 * np.pi * (x - 0.01 * i))
    line.set_data(x, y)
    return (line,)

Finally, we call the animator and display the result.

In [ ]:
# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=20, blit=True)
HTML(anim.to_html5_video())

Can you see some animation? Please describe what you see below. ** **write your answer below (30 pts)** **

(write your answer here)

# 3. PyBullet Rendering test

PyBullet (https://pythonhosted.org/pybullet/) is an open-source physics simulator that is developed by Erwin Coumans and Yunfei Bai. We will use this engine for visualization and simulation. Let's see if we can simulate a robot and visualize it.

This requires us to install a few additional libraries (note: the installation of PyBullet usually takes a few minutes (~6min?). Please contact the instructor if it takes significantly longer).

In [ ]:
%%time
!apt-get install -y xvfb python-opengl ffmpeg
!pip install pybullet==2.6.5
!pip install ffmpeg-python

We import the required libraries again.

In [ ]:
import numpy as np
import random
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

import pybullet as p
import cv2
from PIL import Image
import ffmpeg

We setup the PyBullet scene, including the robot, ground, and camera.

In [ ]:
p.connect(p.DIRECT)
p.resetSimulation()

urdf_root = '/usr/local/lib/python3.10/dist-packages/pybullet_data/'
p.loadURDF(urdf_root + "plane.urdf")
r2d2=p.loadURDF(urdf_root + "r2d2.urdf", [0.0, 0.0, 1.0], [0.924, 0.383, 0.0, 0.0])
assert(p.getNumBodies() == 2)  # Make sure that you have the robot and ground.

p.setGravity(0,0,-10)
pixelWidth = 640
pixelHeight = 360
viewMatrix = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0,0,0],
                                                 distance=4.0,
                                                 yaw=60.0,
                                                 pitch=-10.0,
                                                 roll=0.0,
                                                 upAxisIndex=2)
projectionMatrix = p.computeProjectionMatrixFOV(fov=60,
                                                aspect=pixelWidth / pixelHeight,
                                                nearVal=0.01,
                                                farVal=100)

Now the simulation is good to go! We will simulate the robot for 480 frames (= 2 seconds), render images for every 10 frames.

In [ ]:
frame = 0
for i in range (480):
    p.stepSimulation() # The default timestep is 1/240
    if i % 10 == 0:
      _, _, img, _, _ = p.getCameraImage(pixelWidth, pixelHeight, viewMatrix,projectionMatrix, shadow=1, lightDirection=[1,1,1])
      Image.fromarray(img[:, :, :3]).save('./frame%04d.jpg' % frame)
      frame += 1
ffmpeg.input('./frame*.jpg', pattern_type='glob', framerate=30).output('./output.gif').overwrite_output().run()

We will create an animated image (gif) from the rendered images and show it.

In [ ]:
def show_gif_image(filename):
  video = io.open(filename, 'r+b').read()
  encoded = base64.b64encode(video)
  ipythondisplay.display(HTML(data='''<img src="data:image/gif;base64,{0}"/>'''.format(encoded.decode('ascii'))))

ffmpeg.input('./frame*.jpg', pattern_type='glob', framerate=240.0/10).output('./output.gif').overwrite_output().run()
show_gif_image('./output.gif')

Can you see some animation from PyBullet? Please describe what you see below. ** **write your answer below (30 pts)** **

(write your answer here)